In [125]:
#auto reload 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [126]:
from prep_data import load_data

In [133]:
(train_data, train_dataloader), (val_data, val_dataloader), (bow_en, bow_fr) = load_data(10000,1000)

c:\Users\Alhus\.conda\envs\projetmla\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [139]:
for x in train_dataloader:
    print(x["english"]["idx"])
    break

tensor([[  258, 30000,    26,  ..., 30000,     0,  1027],
        [30000,   223, 30000,  ..., 30000, 30000, 30000],
        [   86,   183,    27,  ...,  3287,   859,    27],
        ...,
        [   62,    39,  1302,  ..., 30000, 30000, 30000],
        [    5,  1095, 30000,  ..., 30000, 30000, 30000],
        [   11,     7,  5914,  ..., 30000, 30000, 30000]], dtype=torch.int16)


BLEU SCORE

1-EXISTANT NLTK

In [ ]:
import nltk 
hypothesis=['Le','chat','se','trouve','sur','le','toit']
reference=['Le','chat','se','trouve','sur','le','toit','bla']
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis) 
print(BLEUscore)


0.8668778997501817


2-SCORE PERSONNALISE

In [ ]:
import collections
import math

def ngram_precision(candidate, reference, n):
    candidate_ngrams = [tuple(candidate[i:i+n]) for i in range(len(candidate)-n+1)]
    reference_ngrams = [tuple(reference[i:i+n]) for i in range(len(reference)-n+1)]

    candidate_ngram_counts = collections.Counter(candidate_ngrams)
    reference_ngram_counts = collections.Counter(reference_ngrams)

    overlap_count = sum((candidate_ngram_counts & reference_ngram_counts).values())
    total_count = max(1, len(candidate_ngrams))

    precision = overlap_count / total_count
    return precision

def bleu_score(candidate, references, weights):
    precisions = []

    for i in range(len(weights)):
        n = i + 1
        precision = ngram_precision(candidate, references, n)
        precisions.append(precision)

    geometric_mean = math.exp(sum(weights[i] * math.log(p) for i, p in enumerate(precisions)))

    brevity_penalty = min(1, len(candidate) / max(len(reference) for reference in references))

    bleu = brevity_penalty * geometric_mean
    return bleu

# Exemple d'utilisation
reference = ['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']
candidate = ['the', 'fast', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']

candidate=['Le','chat','se','trou','s','le','to','bli']
reference=['Le','chat','se','trouve','sur','le','toit']

weights = [0.25,0.25,0.25] # You can adjust the weights based on your preference

score = bleu_score(candidate, reference, weights)
print("BLEU Score:", score)


BLEU Score: 0.392814650900513


In [ ]:
import torch 

#matmul tests: 
batch_size = 2 
Tx = 3 
hidden_size = 4
a = torch.randint(0,10,(batch_size, Tx),dtype=torch.float)
h = torch.randint(0,10,(batch_size, Tx, hidden_size),dtype=torch.float)


In [ ]:
a

tensor([[9., 4., 7.],
        [0., 1., 6.]])

In [ ]:
h.swapaxes(1,2)

tensor([[[0., 0., 3.],
         [7., 1., 2.],
         [9., 5., 0.],
         [5., 6., 9.]],

        [[8., 0., 7.],
         [0., 0., 7.],
         [9., 1., 0.],
         [2., 3., 1.]]])

In [ ]:
h.shape,a.shape

(torch.Size([2, 3, 4]), torch.Size([2, 3]))

In [ ]:
(h.swapaxes(1,2) @ a.unsqueeze(2)).squeeze()

tensor([[ 21.,  81., 101., 132.],
        [ 42.,  42.,   1.,   9.]])